# Advanced pandas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
plt.style.available

['Solarize_Light2',
 '_classic_test_patch',
 'bmh',
 'classic',
 'dark_background',
 'fast',
 'fivethirtyeight',
 'ggplot',
 'grayscale',
 'seaborn',
 'seaborn-bright',
 'seaborn-colorblind',
 'seaborn-dark',
 'seaborn-dark-palette',
 'seaborn-darkgrid',
 'seaborn-deep',
 'seaborn-muted',
 'seaborn-notebook',
 'seaborn-paper',
 'seaborn-pastel',
 'seaborn-poster',
 'seaborn-talk',
 'seaborn-ticks',
 'seaborn-white',
 'seaborn-whitegrid',
 'tableau-colorblind10']

In [5]:
plt.style.use('seaborn-whitegrid')

## 1. Categorical Data

### Background and Motivation

In [6]:
values=pd.Series(['apple','orange','apple','apple']*2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [7]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [8]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [9]:
values=pd.Series([0,1,0,0]*2)

dim=pd.Series(['apple','orange'])

values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [12]:
dim

0     apple
1    orange
dtype: object

In [11]:
dim.take(values) # 0,1,0,0,0,1,0,0: dim's actual position index

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### Categorical Type in pandas

In [19]:
fruits=['apple','orange','apple','apple']*2
N=len(fruits)

df=pd.DataFrame({'fruit':fruits,
                 'basket_id':np.arange(N),
                 'count':np.random.randint(3,15,size=N),
                 'weight':np.random.uniform(0,4,size=N)},
                columns=['basket_id','fruit','count','weight'])

df

,basket_id,fruit,count,weight
0,0,apple,9,2.593364
1,1,orange,14,0.357557
2,2,apple,7,3.401605
3,3,apple,12,0.031502
4,4,apple,12,3.664317
5,5,orange,13,2.758992
6,6,apple,10,3.652222
7,7,apple,4,3.248102


In [20]:
fruit_cat=df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [21]:
c=fruit_cat.values
type(fruit_cat.values)

pandas.core.arrays.categorical.Categorical

In [22]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [23]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [24]:
df['fruit']=df['fruit'].astype('category')
df

,basket_id,fruit,count,weight
0,0,apple,9,2.593364
1,1,orange,14,0.357557
2,2,apple,7,3.401605
3,3,apple,12,0.031502
4,4,apple,12,3.664317
5,5,orange,13,2.758992
6,6,apple,10,3.652222
7,7,apple,4,3.248102


In [25]:
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [26]:
my_categories=pd.Categorical(['foo','bar','baz','foo','bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [30]:
my_categories.codes

array([2, 0, 1, 2, 0], dtype=int8)

In [28]:
categories=['foo','bar','baz']

codes=[0,1,2,0,0,1]

my_cats_2=pd.Categorical.from_codes(codes,categories)

my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [31]:
ordered_cat=pd.Categorical.from_codes(codes,categories,ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [32]:
ordered_cat.codes

array([0, 1, 2, 0, 0, 1], dtype=int8)

In [34]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### Computations with Categories

In [35]:
np.random.seed(123)

draws=np.random.randn(1000)
draws[:5]

array([-1.0856306 ,  0.99734545,  0.2829785 , -1.50629471, -0.57860025])

In [36]:
bins=pd.qcut(draws,4)
bins

[(-3.2319999999999998, -0.685], (0.669, 3.572], (-0.0412, 0.669], (-3.2319999999999998, -0.685], (-0.685, -0.0412], ..., (-0.0412, 0.669], (0.669, 3.572], (-3.2319999999999998, -0.685], (-0.0412, 0.669], (-3.2319999999999998, -0.685]]
Length: 1000
Categories (4, interval[float64]): [(-3.2319999999999998, -0.685] < (-0.685, -0.0412] < (-0.0412, 0.669] < (0.669, 3.572]]

In [37]:
bins=pd.qcut(draws,4,labels=['Q1','Q2','Q3','Q4'])
bins

['Q1', 'Q4', 'Q3', 'Q1', 'Q2', ..., 'Q3', 'Q4', 'Q1', 'Q3', 'Q1']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [38]:
bins.codes[:10]

array([0, 3, 2, 0, 1, 3, 0, 1, 3, 0], dtype=int8)

In [46]:
bins=pd.Series(bins,name='quartile')

result=pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index()

result

,quartile,count,min,max
0,Q1,250,-3.231055,-0.688549
1,Q2,250,-0.683226,-0.043042
2,Q3,250,-0.039329,0.668317
3,Q4,250,0.670512,3.571579


In [47]:
result['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

### Better performance with categoricals

In [52]:
N=10000000
draws=pd.Series(np.random.randn(N))

labels=pd.Series(['foo','bar','baz','qux']*(N//4))

categories=labels.astype('category')

In [53]:
labels.memory_usage()

80000128

In [54]:
categories.memory_usage()

10000320

In [55]:
%%time
_=labels.astype('category')

CPU times: user 375 ms, sys: 35.9 ms, total: 411 ms
Wall time: 411 ms


### Categorical Methods

In [56]:
s=pd.Series(['a','b','c','d']*2)
cat_s=s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [57]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [58]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [59]:
actual_categories=['a','b','c','d','e']

cat_s2=cat_s.cat.set_categories(actual_categories)

cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [60]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [61]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [62]:
cat_s3=cat_s[cat_s.isin(['a','b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [64]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

### Creating dummy variables for modelling

In [65]:
cat_s=pd.Series(['a','b','c','d']*2,dtype='category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [66]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 2. Advanced GroupBy Use

### Group Transforms and "Unwrapped" GroupBys

In [87]:
df=pd.DataFrame({'key':['a','b','c']*4,
                 'value':np.arange(12.)})

df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [88]:
df.groupby(df['key']).mean()

,value
key,
a,4.5
b,5.5
c,6.5


In [89]:
g=df.groupby(df['key']).value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [90]:
df.groupby(df['key']).transform(lambda x:x.mean())

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [91]:
df.groupby(df['key']).transform('mean')

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [92]:
df.groupby(df['key']).transform(lambda x:x*2)

,value
0,0.0
1,2.0
2,4.0
3,6.0
4,8.0
5,10.0
6,12.0
7,14.0
8,16.0
9,18.0


In [93]:
df.groupby(df['key']).transform(lambda x:x.rank(ascending=False))

,value
0,4.0
1,4.0
2,4.0
3,3.0
4,3.0
5,3.0
6,2.0
7,2.0
8,2.0
9,1.0


In [97]:
df.groupby(df['key']).rank(ascending=False)

,value
0,4.0
1,4.0
2,4.0
3,3.0
4,3.0
5,3.0
6,2.0
7,2.0
8,2.0
9,1.0


In [103]:
%%time

def normalize(x):
    return (x-x.mean())/x.std()

df.groupby(df['key']).transform(normalize)

CPU times: user 8.7 ms, sys: 454 µs, total: 9.16 ms
Wall time: 8.86 ms


,value
0,-1.161895
1,-1.161895
2,-1.161895
3,-0.387298
4,-0.387298
5,-0.387298
6,0.387298
7,0.387298
8,0.387298
9,1.161895


In [100]:
df.value.groupby(df['key']).apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [101]:
df.value.groupby(df['key']).transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [104]:
%%time

normalized=(df['value']-df.value.groupby(df['key']).transform('mean'))/df.value.groupby(df['key']).transform('std')

normalized

CPU times: user 2.44 ms, sys: 147 µs, total: 2.58 ms
Wall time: 2.44 ms


0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### Grouped Time Resampling

In [105]:
N=15

times=pd.date_range('2017-05-20 00:00',freq='1min',periods=N)

times

DatetimeIndex(['2017-05-20 00:00:00', '2017-05-20 00:01:00',
               '2017-05-20 00:02:00', '2017-05-20 00:03:00',
               '2017-05-20 00:04:00', '2017-05-20 00:05:00',
               '2017-05-20 00:06:00', '2017-05-20 00:07:00',
               '2017-05-20 00:08:00', '2017-05-20 00:09:00',
               '2017-05-20 00:10:00', '2017-05-20 00:11:00',
               '2017-05-20 00:12:00', '2017-05-20 00:13:00',
               '2017-05-20 00:14:00'],
              dtype='datetime64[ns]', freq='T')

In [106]:
df=pd.DataFrame({'time':times,
                 'value':np.arange(N)})

df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [111]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [128]:
df2=pd.DataFrame({'time':times.repeat(3),
                  'key':np.tile(['a','b','c'],N),
                  'value':np.arange(N*3.)})

df2[:7]
# times.repeat(3)

# np.tile(['a','b','c'],N)

# np.repeat(n) 每个元素重复n次，然后重复下一个元素
# np.tile(list,n) list重复n次

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [129]:
time_key=pd.Grouper(freq='5min')
time_key

TimeGrouper(freq=<5 * Minutes>, axis=0, sort=True, closed='left', label='left', how='mean', convention='e', origin='start_day')

In [130]:
resampled=df2.set_index('time').groupby(['key',time_key]).sum()

resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [131]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0
